<pre>
Copyright 2021-2025 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# since both Jupyter and `isabelle-client` use `asyncio` we need to enable
# nested event loops. We don't need that when using `isabelle-client` from
# Python scripts outside Jupyter
import nest_asyncio

nest_asyncio.apply()

In [2]:
from isabelle_client import start_isabelle_server

# first, we start Isabelle server
server_info, _ = start_isabelle_server(
    name="test", port=9999, log_file="server.log"
)

In [3]:
import logging

from isabelle_client import get_isabelle_client

# then we create Python client to Isabelle server
isabelle = get_isabelle_client(server_info)
!rm -f session.log
# we will log all the messages from the server to a file
isabelle.logger = logging.getLogger()
isabelle.logger.setLevel(logging.INFO)
isabelle.logger.addHandler(logging.FileHandler("session.log"))

In [4]:
# suppose we have an Isabelle theory file
# (probably generated by another Python script)
!cat Example.thy

theory Example
imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
by auto
end

In [5]:
# we can create a session

session_start_result = isabelle.session_start()

In [6]:
# and then we can send this file to the server and get a response
isabelle.use_theories(
    session_id=session_start_result[-1].response_body.session_id,
    theories=["Example"],
    master_dir=".",
)

[TaskOK(response_type=<IsabelleResponseType.OK: 'OK'>, response_body=Task(task='6525884f-07a5-4c5e-9c62-a061b2a7a831'), response_length=None),
 TaskOK(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=Task(task='6525884f-07a5-4c5e-9c62-a061b2a7a831'), response_length=161),
 TaskOK(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=Task(task='6525884f-07a5-4c5e-9c62-a061b2a7a831'), response_length=161),
 TaskOK(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=Task(task='6525884f-07a5-4c5e-9c62-a061b2a7a831'), response_length=163),
 UseTheoriesResponse(response_type=<IsabelleResponseType.FINISHED: 'FINISHED'>, response_body=UseTheoriesResults(ok=True, errors=[], nodes=[NodeResult(node_name='Example.thy', theory_name='Draft.Example', status=NodeStatus(ok=True, total=7, unprocessed=0, running=0, warned=0, failed=0, finished=7, canceled=False, consolidated=True, percentage=100), messages=[Message(kind='writeln', message='theorem \\<forall>x. \\<e

In [7]:
# if we have a session description using ROOT file like this
!rm -rf output
!cat ROOT

session examples = HOL +
  options [document = pdf, document_output = "output"]
  theories
    Example
  document_files
    "root.tex"


In [8]:
# we can build session (`isabelle build`)
isabelle.session_build(dirs=["."], session="examples")

[TaskOK(response_type=<IsabelleResponseType.OK: 'OK'>, response_body=Task(task='473c5cb3-55c0-4d6e-bbf7-85f0f1cb304f'), response_length=None),
 NotificationResponse(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=MessageNotification(kind='writeln', message='Session Pure/Pure', pos=None, task='473c5cb3-55c0-4d6e-bbf7-85f0f1cb304f'), response_length=117),
 NotificationResponse(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=MessageNotification(kind='writeln', message='Session Misc/Tools', pos=None, task='473c5cb3-55c0-4d6e-bbf7-85f0f1cb304f'), response_length=118),
 NotificationResponse(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=MessageNotification(kind='writeln', message='Session HOL/HOL (main)', pos=None, task='473c5cb3-55c0-4d6e-bbf7-85f0f1cb304f'), response_length=122),
 NotificationResponse(response_type=<IsabelleResponseType.NOTE: 'NOTE'>, response_body=MessageNotification(kind='writeln', message='Session Unsorted/examples', p

In [9]:
# the results will appear in `output` folder
!ls output

ls: cannot access 'output': No such file or directory


In [10]:
import asyncio

# or we can issue a free-text command through TCP
# here `asynchronous` argument means 'asynchronous command' as defined
# in section 4.2.6. of Isabelle System manual. `echo` is a synchronous command
# but Python communicates with Isabelle asynchronously as always
asyncio.run(isabelle.execute_command("echo 42", asynchronous=False))

[IsabelleResponse(response_type=<IsabelleResponseType.OK: 'OK'>, response_body=42, response_length=None)]

In [11]:
# we can also shut down the Isabelle server
isabelle.shutdown()

SimpleIsabelleResponse(response_type=<IsabelleResponseType.OK: 'OK'>)

In [12]:
# all our communications with the server
# were written to the log file specified earlier
!cat session.log

809ee2c9-fdaf-4282-bcec-ff58da4c4eab
session_start {"session": "Main"}

OK {"isabelle_id": "4b875a4c83b0", "isabelle_name": "Isabelle2025"}
OK {"task": "0a73844f-47ce-4560-9791-dd137ceb6ace"}
117
NOTE {"kind": "writeln", "message": "Session Pure/Pure", "verbose": "true", "task": "0a73844f-47ce-4560-9791-dd137ceb6ace"}
118
NOTE {"kind": "writeln", "message": "Session Misc/Tools", "verbose": "true", "task": "0a73844f-47ce-4560-9791-dd137ceb6ace"}
122
NOTE {"kind": "writeln", "message": "Session HOL/HOL (main)", "verbose": "true", "task": "0a73844f-47ce-4560-9791-dd137ceb6ace"}
122
NOTE {"kind": "writeln", "message": "Session Doc/Main (doc)", "verbose": "true", "task": "0a73844f-47ce-4560-9791-dd137ceb6ace"}
117
NOTE {"kind": "writeln", "message": "Session Pure/Pure", "verbose": "true", "task": "0a73844f-47ce-4560-9791-dd137ceb6ace"}
118
NOTE {"kind": "writeln", "message": "Session Misc/Tools", "verbose": "true", "task": "0a73844f-47ce-4560-9791-dd137ceb6ace"}
122
NOTE {"kind": "writeln",